In [1]:
import pandas as pd
import numpy as np
from ortools.linear_solver import pywraplp
import random

# Generate Event Data
np.random.seed(42)
num_events = 145
num_concerts = 60
cities = [f"City_{i}" for i in range(70)]
countries = [f"Country_{i}" for i in range(30)]
event_types = ["Festival", "Concert"]
event_fees = np.random.randint(5000, 50000, size=num_events)
event_dates = pd.date_range(start="2024-07-01", end="2024-12-31", periods=num_events).date
events = pd.DataFrame({
    "EventID": range(num_events),
    "City": np.random.choice(cities, num_events),
    "Country": np.random.choice(countries, num_events),
    "EventType": np.random.choice(event_types, num_events),
    "EventFee": event_fees,
    "EventDate": event_dates
})

# Generate Travel Time Data
np.random.seed(42)
travel_time = np.random.randint(1, 11, size=(len(cities), len(cities)))
np.fill_diagonal(travel_time, 0)  # Travel time from a city to itself is 0

# Solver
solver = pywraplp.Solver.CreateSolver('SCIP')
if not solver:
    raise Exception("SCIP solver not found.")

# Decision Variables
x = {}
for i in range(num_events):
    x[i] = solver.BoolVar(f'x[{i}]')

# Objective: Maximize total profit (Event fees only)
objective = solver.Sum([x[i] * events["EventFee"].iloc[i] for i in range(num_events)])
solver.Maximize(objective)

# Constraints

# 1. Exactly 60 concerts
solver.Add(solver.Sum([x[i] for i in range(num_events)]) == num_concerts)

# 2. Minimum 1 rest day between events (incrementally increasing)
min_rest_days = 1  # Start with 1 day

for i in range(num_events):
    for j in range(i + 1, num_events):
        if abs((events["EventDate"].iloc[j] - events["EventDate"].iloc[i]).days) <= min_rest_days:
            solver.Add(x[i] + x[j] <= 1)

# 3. Travel time constraints (incrementally decreasing)
max_travel_time = 12  # Start with 12 hours

for i in range(num_events):
    for j in range(i + 1, num_events):
        if travel_time[cities.index(events["City"].iloc[i])][cities.index(events["City"].iloc[j])] > max_travel_time:
            solver.Add(x[i] + x[j] <= 1)

# 4. Perform in at least 10 different countries (incrementally increasing)
min_countries = 10  # Start with 10 countries

solver.Add(solver.Sum([x[i] * (events["Country"].iloc[i] == country) for i in range(num_events) for country in countries]) >= min_countries)

# Solve the problem
status = solver.Solve()

# Print results
if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    selected_events = []
    for i in range(num_events):
        if x[i].solution_value() > 0:
            selected_events.append(i)
            print(f'Event {events["EventID"].iloc[i]} in {events["City"].iloc[i]}, {events["Country"].iloc[i]} on {events["EventDate"].iloc[i]} selected.')

    selected_countries = events.iloc[selected_events]["Country"].nunique()
    print(f'Total different countries visited: {selected_countries}')
elif status == pywraplp.Solver.FEASIBLE:
    print('A feasible solution was found.')
elif status == pywraplp.Solver.INFEASIBLE:
    print('The problem is infeasible.')
elif status == pywraplp.Solver.UNBOUNDED:
    print('The problem is unbounded.')
else:
    print('The problem does not have an optimal solution.')

# Save the generated events data for reference
events.to_csv('dj_events_basic.csv', index=False)


Objective value = 2209386.0
Event 3 in City_61, Country_4 on 2024-07-04 selected.
Event 7 in City_27, Country_13 on 2024-07-09 selected.
Event 9 in City_29, Country_2 on 2024-07-12 selected.
Event 11 in City_61, Country_0 on 2024-07-14 selected.
Event 17 in City_26, Country_2 on 2024-07-22 selected.
Event 20 in City_36, Country_9 on 2024-07-26 selected.
Event 22 in City_43, Country_25 on 2024-07-28 selected.
Event 23 in City_23, Country_2 on 2024-07-30 selected.
Event 25 in City_31, Country_13 on 2024-08-01 selected.
Event 26 in City_51, Country_23 on 2024-08-03 selected.
Event 29 in City_51, Country_21 on 2024-08-06 selected.
Event 30 in City_11, Country_22 on 2024-08-08 selected.
Event 33 in City_2, Country_9 on 2024-08-11 selected.
Event 37 in City_1, Country_7 on 2024-08-17 selected.
Event 40 in City_18, Country_10 on 2024-08-20 selected.
Event 43 in City_43, Country_6 on 2024-08-24 selected.
Event 46 in City_31, Country_22 on 2024-08-28 selected.
Event 48 in City_54, Country_2 on 